In [2]:
# Mount Google Drive to save model and files
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [3]:
# Upload kaggle.json file
from google.colab import files
files.upload()  # Choose kaggle.json file

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
# Configure kaggle API credentials
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# Download PlantVillage dataset from Kaggle
!kaggle datasets download -d emmarex/plantdisease

In [ ]:
# Unzip dataset
!unzip plantdisease.zip

In [ ]:
# Verify dataset
import os
print(os.listdir("/content/plantvillage/PlantVillage"))

In [ ]:
import os

# Automatically rename files to fix naming issues
crop_folder = "/content/plantvillage/PlantVillage"
for category in os.listdir(crop_folder):
    category_path = os.path.join(crop_folder, category)
    if os.path.isdir(category_path):
        for filename in os.listdir(category_path):
            if " " in filename:
                new_filename = filename.replace(" ", "_")
                os.rename(
                    os.path.join(category_path, filename),
                    os.path.join(category_path, new_filename)
                )
                print(f"Renamed: {filename} ➡️ {new_filename}")

In [ ]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import numpy as np
import json
import os

In [ ]:
# Define dataset directory
dataset_dir = "/content/plantvillage/PlantVillage"

In [ ]:
# Create ImageDataGenerator with Augmentation
datagen = ImageDataGenerator(
    rescale=1./255,         # Normalize pixel values
    validation_split=0.2,   # Split data into 80% train and 20% validation
    rotation_range=20,      # Rotate images for augmentation
    zoom_range=0.2,         # Zoom in/out
    horizontal_flip=True    # Flip images horizontally
)

In [ ]:
# Load training data
train_data = datagen.flow_from_directory(
    dataset_dir,
    target_size=(224, 224),  # Resize images to 224x224
    batch_size=32,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

In [ ]:
# Load validation data
val_data = datagen.flow_from_directory(
    dataset_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

In [ ]:
# Display class indices to verify correct order
print("✅ Class Indices:", train_data.class_indices)

In [ ]:
# Save class indices for future use in deployment
with open('/content/class_indices.json', 'w') as file:
    json.dump(train_data.class_indices, file)
print("✅ Saved class indices to class_indices.json")

In [ ]:
# Build CNN model
model = models.Sequential([
    layers.Input(shape=(224, 224, 3)),

    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(train_data.num_classes, activation='softmax')  # Output layer for multi-class classification
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Show model summary
model.summary()


In [ ]:
# Train the model
history = model.fit(
    train_data,
    epochs=10,  # Increase epochs if needed
    validation_data=val_data
)

In [ ]:
# Plot training and validation accuracy and loss
plt.figure(figsize=(12, 4))

# Plot Accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Plot Loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
# Evaluate model performance
val_loss, val_accuracy = model.evaluate(val_data)
print(f"✅ Validation Accuracy: {val_accuracy * 100:.2f}%")

In [ ]:
# Save trained model
model.save('/content/crop_disease_model.h5')
print("✅ Model saved successfully!")

# Download model and class indices
from google.colab import files
files.download('/content/crop_disease_model.h5')
files.download('/content/class_indices.json')

In [ ]:
from google.colab import files
from tensorflow.keras.preprocessing import image
import numpy as np
import json

# Upload an image to test prediction
uploaded_file = files.upload()  # Upload a leaf image

# Get uploaded image filename
img_filename = list(uploaded_file.keys())[0]
print(f"✅ Uploaded file: {img_filename}")

# Load and preprocess the image
def preprocess_image(img_path):
    """Load and preprocess the image for model prediction."""
    img = image.load_img(img_path, target_size=(224, 224))  # Resize image
    img_array = image.img_to_array(img) / 255.0  # Normalize pixels
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array

# Preprocess the uploaded image
img_array = preprocess_image(img_filename)

In [ ]:
# Load the trained model
from tensorflow.keras.models import load_model

# Load the trained crop disease detection model
model_path = "/content/crop_disease_model.h5"  # Update path if different
model = load_model(model_path)
print("✅ Model loaded successfully!")

# Load class indices from JSON
with open("/content/class_indices.json", "r") as file:
    class_indices = json.load(file)

# Reverse class indices to get class names by index
class_names = {v: k for k, v in class_indices.items()}
print("✅ Class indices loaded and reversed!")

In [ ]:
# Make prediction on uploaded image
predictions = model.predict(img_array)

# Get the predicted class index and label
predicted_class_index = np.argmax(predictions)
predicted_class_label = class_names[predicted_class_index]

# Show prediction result
print(f"✅ Predicted Class: {predicted_class_label.replace('_', ' ')}")